# Custom database creation - Refseq NCBI 16S sequences - as Zaramela did 

In [ ]:
# Parsing Refseq16S Database
## Extract full lineage from NCBI database
library(tidyverse) 

## Loading reference file (linking refseq accession number and taxID); Reference file was obtained by parsing gff3 file provided by NCBI  
RefseqTaxID <- read.delim("data/RefseqTaxID.txt", h=T)

tmp <- readLines("data/rankedlineage.dmp") # File obtained from NCBI refseq16S. It reads the contents of the file and stores it into tmp variable
bac <- grep("Bacteria", tmp) 
tax <- tmp[bac] # Filtering Lines Containing "Bacteria"; 'tax' now contains only the lines from tmp that contain the string "Bacteria"
tax <- tax[1:552981] #last line contain invalid configuration

taxtable = NULL # it initializes an empty data frame named taxtable to store the taxonomic information extracted from the lines

for(i in 1:length(tax)){
    tax1 <- gsub("\t", "", strsplit(tax[i],split = "\\|")[[1]]) # It splits each line using the "|" character as a delimiter and removes any tab characters. To avoid the vertical bar (|) to be interpreted as an escape sequence in your character string (not recognize), I added another `\` to ensure that the vertical bar will be read as a character 
    tax1[tax1 == ""] <- NA # replaces empty strings with NA, indicating missing values
    tax2 <- strsplit(tax1[2],split = " ")[[1]]
    if(length(tax2) >= 2){tax1[3] <- paste0(tax2[1], " ", tax2[2])} 
    if(length(tax2) < 2){tax1[3] <- tax1[3]} # extracts the species name and ensures it is correctly formatted. If the species name consists of two or more words, it combines them into a single string separated by a space, if the species names is one word, it remains unchanged
    tax3 = data.frame(id = tax1[1],
                      name = tax1[2],
                      Species = tax1[3],
                      Genus = tax1[4],
                      Family = tax1[5],
                      Order = tax1[6],
                      Class = tax1[7],
                      Phylum = tax1[8],
                      Kingdom = tax1[9],
                      Domain = tax1[10])
    taxtable <- rbind(taxtable, tax3) # creates a data frame (tax3) containing taxonomic information extracted from each line.
} # this loop takes a long time to run (> 5hours), so no FREAKING OUT, just chill
write.table(taxtable, "referencetable_taxonomy_RefseqNCBI_16S.txt", quote = F, sep = "\t", row.names = F)
